Install libraries

In [ ]:
!pip install langchain langchain-huggingface langchain-community fastembed chromadb transformers torch


2. Import Libraries

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import FastEmbedEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain_community.chat_models import ChatOllama
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import os, getpass

3. Authenticate Hugging Face

In [ ]:
# Prompt for your Hugging Face API key if not already set
if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your Hugging Face API key: ")


4. Load and Split CTSE Lecture Notes

In [ ]:
# Load your lecture notes PDF file
loader = PyPDFLoader("../CTSE_Lecture_Notes.pdf")  # Replace with your file name
pages = loader.load_and_split()

# Split into chunks (important for context-aware retrieval)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=2048,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True,
)
docs = splitter.split_documents(pages)
print(f"Split {len(pages)} pages into {len(docs)} chunks.")

5. Create Embeddings & Vector Store

In [ ]:
persist_directory = "./chroma_langchain_db"

# Use FastEmbed to convert text into vectors
embeddings = FastEmbedEmbeddings()

if os.path.exists(persist_directory):
    # If already exists, load the existing DB
    vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
    print("Loaded existing vector store.")

else:
    # Otherwise, create and save
    vector_store = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    vector_store.persist()
    print("Created and saved new vector store.")


6. Set Up Retriever

In [ ]:
# Retriever with threshold filtering
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 3,
        "score_threshold": 0.5,
    }
)

7. Initialize LLaMA 3 via Ollama

In [ ]:
# Initialize LLaMA 3 via Ollama
llm = ChatOllama(model="tinyllama")

8. Build Retrieval QA Chain

In [ ]:
qa_chain = RetrievalQA.from_llm(llm=llm, retriever=retriever)

9. Define Chatbot Function

In [ ]:
def ask(query: str):
    # Retrieve relevant documents (using updated LangChain method)
    retrieved_docs = retriever.invoke(query)

    if not retrieved_docs:
        print("\n[Info] No relevant documents were retrieved using the score threshold. No answer will be provided.\n")
        return

    # Proceed with the QA chain if relevant documents exist
    result = qa_chain.invoke({"question": query})
    print("\nAnswer:", result["answer"])

    # Optionally print sources
    if "source_documents" in result:
        for doc in result["source_documents"]:
            print("Source:", doc.metadata.get("source", "Unknown"))


10. Batch Querying – Test Multiple Questions

In [ ]:
ask("What is capcut?")